In [2]:
pip install networkx


   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
    --------------------------------------- 0.0/1.7 MB 495.5 kB/s eta 0:00:04
   --- ------------------------------------ 0.1/1.7 MB 1.2 MB/s eta 0:00:02
   ------- -------------------------------- 0.3/1.7 MB 2.1 MB/s eta 0:00:01
   ---------------- ----------------------- 0.7/1.7 MB 3.5 MB/s eta 0:00:01
   ------------------- -------------------- 0.8/1.7 MB 3.4 MB/s eta 0:00:01
   ------------------------------------- -- 1.6/1.7 MB 5.4 MB/s eta 0:00:01
   ---------------------------------------  1.7/1.7 MB 5.7 MB/s eta 0:00:01
   ---------------------------------------  1.7/1.7 MB 5.7 MB/s eta 0:00:01
   ---------------------------------------  1.7/1.7 MB 5.7 MB/s eta 0:00:01
   ---------------------------------------  1.7/1.7 MB 5.7 MB/s eta 0:00:01
   ---------------------------------------  1.7/1.7 MB 5.7 MB/s eta 0:00:01
   --------------------


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
#import các thư viện cần thiết 
import numpy as np 
import pandas as pd
import heapq
import networkx as nx
import matplotlib.pyplot as plt

In [29]:
class Graph: 
    def __init__(self):
        """Khởi tạo đồ thị với danh sách kề rỗng"""
        self.graph = {}
    
    def load_csv (self, filename):
        """
        Đọc đồ thị từ file csv
        Mỗi dòng bao gồm (v_from, v_to, weight)

        """
        graph_csv = pd.read_csv(filename)
        for _, row in graph_csv.iterrows(): 
            #Sử dụng iterrows để truy cập các cột theo tên để không bị lỗi chuyển thành int
            v_from = row['v_from']
            v_to = row['v_to']
            weight = row['weight']

            #Đồ thị VÔ HƯỚNG nên khi thêm cạnh vào đồ thị thì phải thêm cả HAI CHIỀU
            print(f"Adding edge from {v_from} to {v_to} with weight {weight}")  # Debug line
            self.add_edge(v_from, v_to, weight)
    
    def add_edge (self, v_from, v_to, weight):
        """ 
        Thêm cạnh vào danh sách kề của đồ thị 
        Nếu đỉnh chưa tồn tại trong danh sách kề thì tạo một danh sách kề mới
        """
        if v_from not in self.graph:
            self.graph[v_from] = []

        if v_to not in self.graph:
            self.graph[v_to] = []
        
        #Thêm cạnh HAI CHIỀU vì là đồ thị VÔ HƯỚNG 
        self.graph[v_from].append((v_to, weight))
        self.graph[v_to].append((v_from, weight))
    
    def Dijkstra(self, start, end):
        
        dist = {node: (float('inf'), None) for node in self.graph}  
        
        dist[start] = (0, None)
        
        #Danh sách khoảng cách
        list_dist = [(0, start)] #(Khoảng cách, đỉnh)

        #Danh sách đỉnh đã duyệt
        closed = set()

        while list_dist:
            current_dist, current_node = heapq.heappop(list_dist)
            closed.add(current_node)

            #Nếu khoảng cách hiện tại (current_dist) lớn hơn khoảng cách đã duyệt -> bỏ qua nó
            if current_dist > dist[current_node][0]:
                continue
            
            #Duyệt qua các cạnh của đỉnh hiện tại 
            for neighbor, weight in self.graph[current_node]:
                if neighbor not in closed: 
                    #Tính khoảng cách từ đỉnh hiện tại đến láng giềng
                    new_dist = current_dist + weight
            
                    #Nếu khoảng cách mới nhỏ hơn khoảng cách đã tìm thấy -> cập nhật 
                    if new_dist < dist[neighbor][0]:
                        dist[neighbor] = (new_dist, current_node)
                        heapq.heappush(list_dist, (new_dist, neighbor))
        
        return dist
    
    def path(self, distance, start, end):
        #Lưu đường đi từ đích đến điểm bắt đầu 
        path = []
        node = end #node đầu tiên là điểm đích 
        
        while node is not None:
            path.append(node)
            node = distance[node][1]

        #Đảo ngược danh sách để danh sách từ start -> end
        path = path[::-1]

        path_str = '->'.join(map(str, path)) #Thêm mũi tên vào danh sách

        #Chi phí tới điểm đích 
        cost = distance[end][0]

        return path_str, cost

    def print_shortest_path(self, start, end):
        distance = self.Dijkstra(start, end)

        if distance[end][0] == float('inf'): 
            print(f'Không có đường đi từ {start} đến {end}.')

        else:
            path_str, cost = self.path(distance, start, end)
            print(f'Đường đi ngắn nhất từ {start} đến {end} là: {path_str}')
            print(f'Chi phí đường đi là: {cost}')

In [ ]:
graph = Graph()
graph.load_csv(r'C:\Users\ASUS\Documents\GitHub\Final-LTPTDL\Graph.csv')

Adding edge from A to C with weight 9
Adding edge from A to F with weight 20
Adding edge from A to D with weight 7
Adding edge from A to E with weight 13
Adding edge from C to H with weight 6
Adding edge from D to H with weight 8
Adding edge from D to E with weight 4
Adding edge from E to K with weight 4
Adding edge from E to I with weight 3
Adding edge from F to I with weight 6
Adding edge from F to G with weight 4
Adding edge from H to K with weight 5
Adding edge from K to B with weight 6
Adding edge from I to K with weight 9
Adding edge from I to B with weight 5


In [32]:
start = 'B'
end = 'F'

graph.print_shortest_path(start, end)

Đường đi ngắn nhất từ B đến F là: B->I->F
Chi phí đường đi là: 11
